# 함수 정의하기

인스타그램 검색 결과 URL을 만들어서 접속하기 (insta_searching 함수 만들기)

In [1]:
#함수 작성
def insta_searching(word):  #word라는 매개변수를 받는 insta_searching 이라는 함수 생성
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [33]:
#셀레니움으로 특정 url에 접속하기
from selenium import webdriver
driver = webdriver.Chrome("C:\\Users\\KIM EUNKI\\anaconda3\\chromedriver.exe")
 #크롬드라이버 실행되게 하기

word = '제주도맛집'
url = insta_searching(word) #만들어 놓은 함수 사용
driver.get(url) #get 함수 사용

로그인 하기

In [ ]:
time.sleep(3)

# 로그인 하기
login_section = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button'
driver.find_element_by_xpath(login_section).click()
time.sleep(2)


elem_login = driver.find_element_by_name("username")
elem_login.clear()
elem_login.send_keys('인스타그램 계정')

elem_login = driver.find_element_by_name('password')
elem_login.clear()
elem_login.send_keys('비밀번호')

time.sleep(1)

xpath = """//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button"""
driver.find_element_by_xpath(xpath).click()

time.sleep(4)

첫번째 게시글 열기 (selet_first 함수 만들기)

In [24]:
#열린 크롬으로 개발자 도구 활용하여 첫번째 게시물 태그 확인 (<div class="_9AhH0"></div>)
#첫번째 게시물 찾아 클릭 함수 만들기

import time

def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0') #find_element_by_css_selector 함수를 사용해 요소 찾기
    first.click()
    time.sleep(3) #로딩을 위해 3초 대기

게시글 정보 가져오기 (get_content 함수 만들기)

In [21]:
#본문 내용, 작성 일시, 위치 정보 및 해시태그(#) 추출

In [45]:
import re

def get_content(driver):
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 2. 본문 내용 가져오기
    try:                                         #여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text #첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
        
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) #content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    
    # 4. 작성 일자 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div._vJRqr > span')[0].text
    except:
        like = 0
    
    # 6. 수집한 정보 저장하기
    data = [content, date, like, tags]
    return data

다음 게시글 열기 (move_next 함수 만들기)

In [34]:
def move_next(driver):
    right = driver.find_element_by_css_celector('a.65Bje.coreSpriteRightPaginationArrow') #다음버튼 xpath: /html/body/div[4]/div[1]/div/div/a
    right.click()
    time.sleep(3)


☆ 여러 게시글 정보 수집하기 (만든 함수들로 코딩하기)

In [49]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬으로 인스타그램 - '제주도맛집' 검색
driver = webdriver.Chrome("C:\\Users\\KIM EUNKI\\anaconda3\\chromedriver.exe")
word = '제주도맛집'
url = insta_searching(word)
driver.get(url) 
time.sleep(4)

# 2. 로그인 하기
login_section = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button'
driver.find_element_by_xpath(login_section).click()
time.sleep(3)


elem_login = driver.find_element_by_name("username")
elem_login.clear()
elem_login.send_keys('01031671816')

elem_login = driver.find_element_by_name('password')
elem_login.clear()
elem_login.send_keys('wkanfthl42#')

time.sleep(1)

xpath = """//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button"""
driver.find_element_by_xpath(xpath).click()

time.sleep(4)

xpath1 = """//*[@id="react-root"]/section/main/div/div/div/div/button"""
driver.find_element_by_xpath(xpath1).click()
time.sleep(4)

# 3. 검색페이지 접속하기
driver.get(url)
time.sleep(4)

# 4. 첫번째 게시글 열기
select_first(driver)

# 5. 비어있는 변수 만들기
results = []

# 여러 게시물 크롤링하기
target = 5 #크롤링할 게시물 수
for i in range(target):
    data = get_content(driver) #게시물 정보 가져오기
    results.append(data)
    move_next = """/html/body/div[4]/div[1]/div/div/a"""
    driver.find_element_by_xpath(move_next).click()
    time.sleep(4)
    
print(results[:2])

[['#제주여행 필수코스 #흑돼지박스 포장하고 바로옆 섭지해변에서 성산일출봉을 바라보며 먹는 제주도시락😋..✔섭지해변 도보2분 외 광치기해변 월정리해변 세화해변 성산일출봉 만장굴 비자림 백약이오름 아쿠아플라넷 등등 주요관광지가 근처에 위치해있어 어디가서먹어도👍..✔미리 전화주문 후 찾으러가면 기다리지않고 포장해 갈수있어요~..✔성산읍 신양로100 흑돼지박스 성산점 064)783-0519✔안덕면 사계남로168 흑돼지박스 산방산점 064)794-0514', '2020-06-15', 0, ['#제주여행', '#흑돼지박스']], ['#제주가성비맛집 정식, 보말수제비, 비빔밥이 8,000원씩그렇다고 부실한 반찬이아닌 김치부터 모든 밑반찬을 직접만드심👍#보말칼국수 대신 보말수제비~ 진정한 보말맛집😋도민들사이에서만 쉬쉬거리며 유명한 #제주도민맛집.🔹️상호:#보리밥과수제비.🔹️아침식사가능예약문의 📞064-721-1239.🔹️오전9~오후5시까지 영업 (재료소진시 조기마감).🔹️제주시 가령로 61-1', '2020-06-15', 0, ['#제주가성비맛집', '#보말칼국수', '#제주도민맛집.🔹️상호:', '#보리밥과수제비.🔹️아침식사가능예약문의']]]


In [50]:
#500번 반복은 오래걸리기 때문에 일단 5번만 해서 2개만 출력해 보았다.